In [183]:
import datetime
import json
import pandas as pd

input_file = 'plow20180101.csv'
data = pd.read_csv(input_file)

data.drop(columns=set(data.keys()) - set(['truck_name', 'date_fixed', 'longitude', 'latitude']), inplace=True)

In [184]:
def fix_date_time(dt: str) -> int:
    return int(datetime.datetime(int(dt[:4]), int(dt[5:7]),
                                 int(dt[8:10]), int(dt[10:12]),
                                 int(dt[13:15]), int(dt[16:18])).timestamp())

data['date_fixed'] = data['date_fixed'].apply(fix_date_time)

In [185]:
start_time = min(data['date_fixed'])

data['date_fixed'] = data['date_fixed'].apply((lambda a: a - start_time))

In [186]:
max_lat, min_lat = data['latitude'].max(),  data['latitude'].min()
max_lon, min_lon = data['longitude'].max(), data['longitude'].min()

d_lat = max_lat - min_lat
d_lon = max_lon - min_lon
m_d = max(d_lat, d_lon)

data['latitude']  = data['latitude'].apply(lambda n: (n - min_lat) / m_d)
data['longitude'] = data['longitude'].apply(lambda n: (n - min_lon) / m_d)

In [187]:
grouped = {key: data.loc[value].drop(columns=['truck_name']) for key, value in data.groupby('truck_name').groups.items()}

In [188]:
grouped_json = {key: value.to_dict(orient='index') for key, value in grouped.items()}
grouped_json = [{int(value['date_fixed']): [value['longitude'], value['latitude']] for key, value in entry.items()} for truck, entry in grouped_json.items()]
grouped_json = [dic for dic in grouped_json if len(dic) > 1]

with open(input_file + '.js', 'w') as file:
    file.write('var a = ')
    json.dump(grouped_json, file, indent=0, separators=(',', ': '))

In [189]:
grouped_json

[{46: [0.6842443328745494, 0.5242722586632447],
  106: [0.6843126883654792, 0.5244557392031732],
  166: [0.6847710300052797, 0.5244722884458296],
  226: [0.6837291482407996, 0.5241967078346564],
  235: [0.6833089418075498, 0.5237541959349026],
  245: [0.6817367653502096, 0.5219503303600813],
  274: [0.6707818964367501, 0.5222143980028943],
  275: [0.6707631886099847, 0.5222733995865254],
  334: [0.6708783136554557, 0.5224640754431507],
  394: [0.6707617495444367, 0.5223115347571858],
  405: [0.6716215898325425, 0.5222554113321053],
  407: [0.6720101369179102, 0.5221028706654659],
  442: [0.6801135021395369, 0.5202666265468052],
  484: [0.6485123935390164, 0.512461147423715],
  499: [0.6429029249173915, 0.5141664369030267],
  518: [0.6422359191305019, 0.5138318547211572],
  555: [0.6444952498472408, 0.4938914778665943],
  586: [0.6444844569059387, 0.49343097769519095],
  642: [0.6004649524654974, 0.48053337354313796],
  684: [0.5991503682174123, 0.4802729030823345],
  736: [0.5573707643